In [1]:
%pylab inline
import os
import trako as TKO

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings("ignore") # hide vtk.util warning

In [3]:
DATASETNAME = 'WholeBrain'
DATADIR = '/home/d/DATA/Dropbox/'+DATASETNAME+'/'
SUBJECTS = ['']
GROUP = ['']

# Tkompare!

In [4]:
OUTPUTDIR = '/home/d/DATA/Dropbox/'+DATASETNAME+'_TKO/'
diffs = {}
for S in SUBJECTS:
    for G in GROUP:
        CURRENTDIR = os.path.join(DATADIR, S, G)
        CURRENTOUTDIR = os.path.join(OUTPUTDIR, S, G)
        files = os.listdir(CURRENTDIR)
        
        for f in files:
            if not f.endswith('vtk'):
                continue
            
            vtpfile = os.path.join(CURRENTDIR, f)
            tkofile = os.path.join(CURRENTOUTDIR, f.replace('vtk', 'tko'))
            
            # load original
            poly_a = TKO.Util.loadvtp(vtpfile)
            
#             try:
                # untrakofy TKO file
            decoded = TKO.Decoder.toVtp(tkofile, verbose=False)
            print(decoded)
            poly_b = TKO.Util.loadvtp(decoded, fromfile=False)
#             except:
#                 continue
                
            # compare points and lines
            (min_a, max_a, mean_a, std_a), dist_a = TKO.Util.normalized_error(poly_a['points'], 
                                                                   poly_b['points'])
            if not 'points' in diffs:
                diffs['points'] = []
            diffs['points'].append(mean_a)
            
            (min_a, max_a, mean_a, std_a), dist_a = TKO.Util.normalized_error(poly_a['lines'], 
                                                                   poly_b['lines'])
            if not 'lines' in diffs:
                diffs['lines'] = []
            diffs['lines'].append(mean_a)
            
            # compare scalars
            for i,s in enumerate(poly_a['scalar_names']):
                (min_a, max_a, mean_a, std_a), dist_a = TKO.Util.normalized_error(poly_a['scalars'][i], 
                                                                                  poly_b['scalars'][i])
                if not s in diffs:
                    diffs[s] = []
                diffs[s].append(mean_a)
                
            # compare properties
            for i,p in enumerate(poly_a['property_names']):
                (min_a, max_a, mean_a, std_a), dist_a = TKO.Util.normalized_error(poly_a['properties'][i], 
                                                                                  poly_b['properties'][i])
                if not p in diffs:
                    diffs[p] = []
                diffs[p].append(mean_a)
                
            
            print('Compared', f, np.mean(diffs['points']))
            

vtkPolyData (0x561fe4d72430)
  Debug: Off
  Modified Time: 426
  Reference Count: 1
  Registered Events: (none)
  Information: 0x561fe4d579c0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 0
  Field Data:
    Debug: Off
    Modified Time: 320
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 5650084
  Number Of Cells: 153537
  Cell Data:
    Debug: Off
    Modified Time: 323
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x561fe4d23b80)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x561fe4d2b260
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
    Normals: (none)
 

# Plots

In [5]:
def reject_outliers(data, m=2):
    data = np.array(data)
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [6]:
import plotly.graph_objects as go

In [9]:
fig = go.Figure()

data = []
errors = []
names = []

for d in diffs:
    
    values = diffs[d]#reject_outliers(np.nan_to_num(diffs[d]))
    data.append(np.nan_to_num(np.mean(values)))
    errors.append(np.nan_to_num(np.std(values)))
    names.append(d)
    
fig.add_trace(go.Bar(x=data, y=names,
                marker_color = 'lightslategray',    orientation='h',
                error_x=dict(type='data', array=errors))
)

fig.update_layout(yaxis_title="per-vertex / per-fiber Data")
fig.update_layout(xaxis_title="Mean Relative Loss per Value")
    
fig.update_layout(xaxis_type="log",xaxis=dict(tickformat=".5f"))
fig.show()
